In [1]:
import numpy as np
from ortools.sat.python import cp_model
import time

puzzle_size = 5

model = cp_model.CpModel()

start_numbers = {
    5: 4,
    9: 2,
    12: 4,
    19: 4
}

start_numbers = {2: 5, 8: 3, 17: 1}

start_numbers.keys()

grid = np.array([model.NewIntVar(start_numbers.get(i, 1), 
                                 start_numbers.get(i, 5), 
                                 f'{i}') for i in range(puzzle_size ** 2)])

grid = grid.reshape((puzzle_size, -1))

grid

grid.shape

# need to add constraint to all rows and column -- all different
for row in grid:
    print(row)
    model.AddAllDifferent(row)

for column in grid.T:
    print(column)
    model.AddAllDifferent(column)
    

grid[0,1]

# Create inequality constraints  
# data format as tuples, 
# * `[0]` = is on the lower side
# * `[1]` = is on the higher side
# 
# as 1D coordinates
ineqs = [(1,0), (3,2), (4,3), (18,19), (20,21), (21,22)]

# as 2D coordinates
ineqs = [
    ((0,1), (0,0)),
    ((0,3), (0,2)),
    ((0,4), (0,3)),
    ((3,3), (3,4)),
    ((4,0), (4,1)),
    ((4,1), (4,2))
    ]

ineqs = [
        ((0,0), (1,0)),
        ((1,2), (1,3)),
        ((1,1), (2,1)),
        ((1,2), (2,2)),
        ((1,4), (2,4)),
        ((2,3), (2,2)),
        ((2,0), (3,0)),
        ((2,3), (2,4)),
        ((3,1), (3,0)),
        ((3,3), (3,4)),
        ((3,1), (4,1)),
        ((4,1), (4,2)),
        ((4,4), (4,3))
        ]

[0(1..5) 1(1..5) 2(5) 3(1..5) 4(1..5)]
[5(1..5) 6(1..5) 7(1..5) 8(3) 9(1..5)]
[10(1..5) 11(1..5) 12(1..5) 13(1..5) 14(1..5)]
[15(1..5) 16(1..5) 17(1) 18(1..5) 19(1..5)]
[20(1..5) 21(1..5) 22(1..5) 23(1..5) 24(1..5)]
[0(1..5) 5(1..5) 10(1..5) 15(1..5) 20(1..5)]
[1(1..5) 6(1..5) 11(1..5) 16(1..5) 21(1..5)]
[2(5) 7(1..5) 12(1..5) 17(1) 22(1..5)]
[3(1..5) 8(3) 13(1..5) 18(1..5) 23(1..5)]
[4(1..5) 9(1..5) 14(1..5) 19(1..5) 24(1..5)]


In [2]:
grid[ineqs[1][1]]

8(3)

In [3]:
grid[0,2]

2(5)

In [4]:
for i in ineqs:
    model.Add(grid[i[0]] < grid[i[1]])

In [5]:
solver = cp_model.CpSolver()

In [6]:
for row in grid:
    print(row.tolist())

[0(1..5), 1(1..5), 2(5), 3(1..5), 4(1..5)]
[5(1..5), 6(1..5), 7(1..5), 8(3), 9(1..5)]
[10(1..5), 11(1..5), 12(1..5), 13(1..5), 14(1..5)]
[15(1..5), 16(1..5), 17(1), 18(1..5), 19(1..5)]
[20(1..5), 21(1..5), 22(1..5), 23(1..5), 24(1..5)]


In [7]:
row_sep = list('+---+---+---+---+---+')

row_subs = [2, 6, 10, 14, 18]
row_sep[row_subs[0]] = u'\u22C0'

In [8]:
''.join(row_sep)

'+-⋀-+---+---+---+---+'

In [9]:
u'\u22C0'

'⋀'

In [10]:
u'\u22C1'

'⋁'

In [11]:
vals = [1, 2, 3, 4, 5]
['|', ' '] + list(' | '.join(str(x) for x in vals)) + [' ', '|']
col_subs = [4, 8, 12, 16]

In [40]:
class DiagramPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables, ineqs):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__ineqs = ineqs
    
    def OnSolutionCallback(self):
        self.__solution_count += 1
        
        # define inequality symbols
        tb = '˄' # u'\u22C0' # ⋀
        bt = '˅' # u'\u22C1' # ⋁
        lr = '<'
        rl = '>'
        
        ineqs = self.__ineqs.copy()
        
        print('+---+---+---+---+---+')
        
        for i, row in enumerate(self.__variables):
            # if i == 1: break
            scope_ineqs = [ineq for ineq in ineqs if ineq[0][0] == i or ineq[1][0] == i]
            
            row = row.tolist()
            vals = [self.Value(val) for val in row]
            
            col_sep = ['|', ' '] + list(' | '.join(str(x) for x in vals)) + [' ', '|']
            col_subs = [4, 8, 12, 16]
            
            row_sep = list('+---+---+---+---+---+')
            row_subs = [2, 6, 10, 14, 18]
            
            # if i == 2: break
                
            for ineq in scope_ineqs:
                lower = ineq[0]
                higher = ineq[1]
                
                
                
                # if i == 0: print(lower, higher)
                
                if lower[0] == higher[0]: # if in the same row
                    if lower[1] > higher[1]: # if lower end is to the right
                        col_sep[col_subs[min(lower[1], higher[1])]] = rl
                    elif lower[1] < higher[1]: # if lower end is to the left
                        col_sep[col_subs[min(lower[1], higher[1])]] = lr
                    
                elif i != 4 and lower[1] == higher[1]: # if in the same column, don't need for final row
                    if lower[0] > higher[0]: # if the lower end is on the bottom column
                        row_sep[row_subs[lower[1]]] = bt
                    elif lower[0] < higher[0]: # if the lower end is on top column
                        row_sep[row_subs[lower[1]]] = tb
                
                # remove entry at end of loop
                ineqs.remove(ineq)
                       
            print(''.join(col_sep))
            print(''.join(row_sep))
        
    
    def SolutionCount(self):
        return self.__solution_count

In [41]:
solution_printer = DiagramPrinter(grid, ineqs)
start = time.time()
status = solver.SearchForAllSolutions(model, solution_printer)
end = time.time()
print(f'Time elapsed: {round(end - start, 4)}')
print('Solutions found : %i' % solution_printer.SolutionCount())

+---+---+---+---+---+
| 4 | 1 | 5 | 2 | 3 |
+-˄-+---+---+---+---+
| 5 | 4 | 2 < 3 | 1 |
+---+-˄-+-˄-+---+-˄-+
| 2 | 5 | 3 > 1 < 4 |
+-˄-+---+---+---+---+
| 3 > 2 | 1 | 4 < 5 |
+---+-˄-+---+---+---+
| 1 | 3 < 4 | 5 > 2 |
+---+---+---+---+---+
Time elapsed: 0.003
Solutions found : 1


In [30]:
'\u2c5'

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-4: truncated \uXXXX escape (<ipython-input-30-8e8f487bee85>, line 1)

In [ ]:
'^'

In [32]:
'˅'

SyntaxError: bytes can only contain ASCII literal characters. (<ipython-input-32-d5acc3d5c568>, line 1)

In [48]:
ineqs

[((0, 0), (1, 0)),
 ((1, 2), (1, 3)),
 ((1, 1), (2, 1)),
 ((1, 2), (2, 2)),
 ((1, 4), (2, 4)),
 ((2, 3), (2, 2)),
 ((2, 3), (2, 4)),
 ((2, 0), (3, 0)),
 ((3, 1), (3, 0)),
 ((3, 3), (3, 4)),
 ((3, 1), (4, 1)),
 ((4, 1), (4, 2)),
 ((4, 4), (4, 3))]

In [17]:
print(u'\u22C0')
print(u'\u22C1')
print('<')
print('>')

⋀
⋁
<
>


In [27]:
import tkinter as tk
from tkinter import Entry
from tkinter import Tk
from tkinter import Button
from tkinter import END

In [28]:
class SampleApp(tk.Tk):
    ### create larger grid with space for < or > between each cell
    def __init__(self):
        tk.Tk.__init__(self)
        self.title('Futoshiki Puzzle Setup')
        self.entry_grid(grid)
        self.button = Button(self, text='Submit', command=self.on_submit, width = 6)
        self.button.grid(row = 5, columnspan = 5)
        
        self.button = Button(self, text='<', command=self.on_button, width = 2)
        self.button.grid(row = 5, columnspan = 1, column = 0)
        
        self.button = Button(self, text='>', command=self.on_button, width = 2)
        self.button.grid(row = 5, columnspan = 1, column = 1)
        
    def entry_grid(self, grid):
        self.entries = []
        for index, _ in np.ndenumerate(grid):
            entry = Entry(self, width = 8, bd = 2, justify = 'center')
            entry.grid(row = index[0], column = index[1])
            self.entries.append(entry)
            
    def generate_ineqs(self):
        pass

    def on_submit(self):
        self.values = np.array([ent.get() for ent in self.entries]).reshape(grid.shape)
        self.destroy()
        
    def on_button(self):
        self.button.configure(relief=tk.SUNKEN)

In [29]:
w = SampleApp()

In [25]:
w.button.__dict__

NameError: name 'w' is not defined

In [30]:
w.mainloop()

In [415]:
w.values

AttributeError: '_tkinter.tkapp' object has no attribute 'values'

In [21]:
import tkinter as tk
import random

class App(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.canvas = tk.Canvas(self, width=500, height=500, borderwidth=0, highlightthickness=0)
        self.canvas.pack(side="top", fill="both", expand="true")
        self.rows = 100
        self.columns = 100
        self.cellwidth = 25
        self.cellheight = 25

        self.rect = {}
        self.oval = {}
        for column in range(20):
            for row in range(20):
                x1 = column*self.cellwidth
                y1 = row * self.cellheight
                x2 = x1 + self.cellwidth
                y2 = y1 + self.cellheight
                self.rect[row,column] = self.canvas.create_rectangle(x1,y1,x2,y2, fill="blue", tags="rect")
                self.oval[row,column] = self.canvas.create_oval(x1+2,y1+2,x2-2,y2-2, fill="blue", tags="oval")

        self.redraw(1000)

    def redraw(self, delay):
        self.canvas.itemconfig("rect", fill="blue")
        self.canvas.itemconfig("oval", fill="blue")
        for i in range(10):
            row = random.randint(0,19)
            col = random.randint(0,19)
            item_id = self.oval[row,col]
            self.canvas.itemconfig(item_id, fill="green")
        self.after(delay, lambda: self.redraw(delay))

In [22]:
app = App()
app.mainloop()